# Machine Learning Model for RC Beam Frequency Prediction
## Part 2: Model Training and Validation

**Objective:** Develop and validate machine learning models to predict natural frequencies of corroded RC beams.

**Author:** Master's Research Project  
**Date:** November 2025

## 1. Setup and Imports

In [1]:
!pip install catboost
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
import catboost
print(catboost.__version__)


  Using cached ipywidgets-8.1.8-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.15-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.16-py3-none-any.whl.metadata (20 kB)
Using cached ipywidgets-8.1.8-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.16-py3-none-any.whl (914 kB)
Using cached widgetsnbextension-4.0.15-py3-none-any.whl (2.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
import xgboost as xgb
from catboost import CatBoostRegressor
import shap
import joblib

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✓ All libraries imported successfully")

ModuleNotFoundError: No module named 'xgboost'

## 2. Configure Logging System

In [ ]:
# Setup logging
log_dir = Path('simulation/logs')
log_dir.mkdir(parents=True, exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_dir / 'ml_training.log'),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)
logger.info("=" * 80)
logger.info("ML MODEL TRAINING INITIALIZED")
logger.info("=" * 80)

## 3. Data Loading and Initial Exploration

In [ ]:
# Load dataset
data_path = Path('simulation/data/beam_vibration_dataset.csv')
logger.info(f"Loading dataset from: {data_path}")

try:
    df = pd.read_csv(data_path)
    logger.info(f"✓ Dataset loaded successfully: {df.shape[0]} samples, {df.shape[1]} features")
except Exception as e:
    logger.error(f"Failed to load dataset: {e}")
    raise

# Display basic information
print("\n" + "="*80)
print("DATASET OVERVIEW")
print("="*80)
print(f"\nShape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst 5 rows:")
df.head()

In [ ]:
# Statistical summary
print("\n" + "="*80)
print("STATISTICAL SUMMARY")
print("="*80)
df.describe()

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
if missing_values.any():
    logger.warning(f"Missing values detected:\n{missing_values[missing_values > 0]}")
else:
    logger.info("✓ No missing values detected")

# Data types
print("\nData Types:")
print(df.dtypes)

## 4. Exploratory Data Analysis (EDA)

In [ ]:
# Create output directory for figures
output_dir = Path('simulation/outputs/ml_figures')
output_dir.mkdir(parents=True, exist_ok=True)
logger.info(f"Output directory created: {output_dir}")

In [ ]:
# Distribution of input features
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Distribution of Input Parameters', fontsize=16, fontweight='bold')

features = ['Length', 'Width', 'Depth', 'Conc_Strength', 'Damage_Severity']
for idx, feature in enumerate(features):
    ax = axes[idx // 3, idx % 3]
    ax.hist(df[feature], bins=30, edgecolor='black', alpha=0.7)
    ax.set_xlabel(feature, fontsize=12)
    ax.set_ylabel('Frequency', fontsize=12)
    ax.set_title(f'{feature} Distribution', fontsize=13)
    ax.grid(alpha=0.3)

# Damage type distribution
ax = axes[1, 2]
df['Damage_Type'].value_counts().plot(kind='bar', ax=ax, edgecolor='black')
ax.set_xlabel('Damage Type', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
ax.set_title('Damage Type Distribution', fontsize=13)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'parameter_distributions.png', dpi=300, bbox_inches='tight')
plt.show()
logger.info("✓ Parameter distribution plot saved")

In [ ]:
# Correlation analysis
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix of Features', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig(output_dir / 'correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()
logger.info("✓ Correlation matrix saved")

# Log key correlations with target
print("\nCorrelations with Mode 1 Frequency:")
print(correlation_matrix['Freq_Mode_1'].sort_values(ascending=False))

In [ ]:
# Damage severity vs frequency
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Mode 1
axes[0].scatter(df['Damage_Severity'], df['Freq_Mode_1'], alpha=0.5, s=20)
axes[0].set_xlabel('Damage Severity (%)', fontsize=12)
axes[0].set_ylabel('Mode 1 Frequency (Hz)', fontsize=12)
axes[0].set_title('Damage Severity vs Mode 1 Frequency', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)

# Mode 2
axes[1].scatter(df['Damage_Severity'], df['Freq_Mode_2'], alpha=0.5, s=20, color='orange')
axes[1].set_xlabel('Damage Severity (%)', fontsize=12)
axes[1].set_ylabel('Mode 2 Frequency (Hz)', fontsize=12)
axes[1].set_title('Damage Severity vs Mode 2 Frequency', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'damage_vs_frequency.png', dpi=300, bbox_inches='tight')
plt.show()
logger.info("✓ Damage vs frequency plot saved")

## 5. Data Preprocessing

In [ ]:
# Prepare features and target
logger.info("Preparing features and target variables")

# Define features and targets
feature_cols = ['Length', 'Width', 'Depth', 'Conc_Strength', 'Damage_Severity']
target_col = 'Freq_Mode_1'

X = df[feature_cols]
y = df[target_col]

logger.info(f"Features shape: {X.shape}")
logger.info(f"Target shape: {y.shape}")
logger.info(f"Feature columns: {feature_cols}")

print(f"\nFeatures (X): {X.shape}")
print(f"Target (y): {y.shape}")

In [ ]:
# Train-test split with stratification
logger.info("Splitting data into train and test sets (80/20)")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, shuffle=True
)

logger.info(f"Train set: {X_train.shape[0]} samples")
logger.info(f"Test set: {X_test.shape[0]} samples")

print(f"\nTraining samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

In [ ]:
# Feature scaling
logger.info("Applying StandardScaler to features")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logger.info("✓ Feature scaling completed")
print("\n✓ Data preprocessing completed")

## 6. Model Training and Evaluation

In [ ]:
# Dictionary to store models and results
models = {}
results = []

def evaluate_model(name, model, X_tr, y_tr, X_te, y_te, use_scaling=True):
    """Train and evaluate a model with comprehensive metrics."""
    logger.info(f"\n{'='*80}")
    logger.info(f"Training {name}")
    logger.info(f"{'='*80}")
    
    # Select scaled or unscaled data
    X_train_data = X_tr if use_scaling else X_train
    X_test_data = X_te if use_scaling else X_test
    
    # Train model
    model.fit(X_train_data, y_tr)
    logger.info(f"✓ {name} training completed")
    
    # Predictions
    y_train_pred = model.predict(X_train_data)
    y_test_pred = model.predict(X_test_data)
    
    # Calculate metrics
    train_mae = mean_absolute_error(y_tr, y_train_pred)
    train_rmse = np.sqrt(mean_squared_error(y_tr, y_train_pred))
    train_r2 = r2_score(y_tr, y_train_pred)
    
    test_mae = mean_absolute_error(y_te, y_test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_te, y_test_pred))
    test_r2 = r2_score(y_te, y_test_pred)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train_data, y_tr, cv=5, scoring='r2')
    cv_mean = cv_scores.mean()
    cv_std = cv_scores.std()
    
    # Log results
    logger.info(f"Train MAE: {train_mae:.4f}, RMSE: {train_rmse:.4f}, R²: {train_r2:.4f}")
    logger.info(f"Test MAE: {test_mae:.4f}, RMSE: {test_rmse:.4f}, R²: {test_r2:.4f}")
    logger.info(f"CV R² Score: {cv_mean:.4f} ± {cv_std:.4f}")
    
    # Store results
    results.append({
        'Model': name,
        'Train_MAE': train_mae,
        'Train_RMSE': train_rmse,
        'Train_R2': train_r2,
        'Test_MAE': test_mae,
        'Test_RMSE': test_rmse,
        'Test_R2': test_r2,
        'CV_R2_Mean': cv_mean,
        'CV_R2_Std': cv_std
    })
    
    return model, y_test_pred

print("Model evaluation function defined")

### 6.1 Linear Regression (Baseline)

In [ ]:
lr_model = LinearRegression()
lr_trained, lr_predictions = evaluate_model(
    'Linear Regression', lr_model, X_train_scaled, y_train, X_test_scaled, y_test
)
models['Linear Regression'] = (lr_trained, lr_predictions)

### 6.2 Random Forest Regressor

In [ ]:
rf_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=0
)
rf_trained, rf_predictions = evaluate_model(
    'Random Forest', rf_model, X_train_scaled, y_train, X_test_scaled, y_test
)
models['Random Forest'] = (rf_trained, rf_predictions)

### 6.3 XGBoost Regressor

In [ ]:
xgb_model = xgb.XGBRegressor(
    n_estimators=200,
    max_depth=8,
    learning_rate=0.1,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbosity=0
)
xgb_trained, xgb_predictions = evaluate_model(
    'XGBoost', xgb_model, X_train_scaled, y_train, X_test_scaled, y_test
)
models['XGBoost'] = (xgb_trained, xgb_predictions)

### 6.4 CatBoost Regressor

In [ ]:
cb_model = CatBoostRegressor(
    iterations=200,
    depth=8,
    learning_rate=0.1,
    random_state=RANDOM_STATE,
    verbose=False
)
cb_trained, cb_predictions = evaluate_model(
    'CatBoost', cb_model, X_train_scaled, y_train, X_test_scaled, y_test
)
models['CatBoost'] = (cb_trained, cb_predictions)

### 6.5 Support Vector Regression

In [ ]:
svr_model = SVR(kernel='rbf', C=100, gamma='scale', epsilon=0.1)
svr_trained, svr_predictions = evaluate_model(
    'SVR', svr_model, X_train_scaled, y_train, X_test_scaled, y_test
)
models['SVR'] = (svr_trained, svr_predictions)

## 7. Model Comparison and Visualization

In [ ]:
# Create results dataframe
results_df = pd.DataFrame(results)
print("\n" + "="*80)
print("MODEL COMPARISON RESULTS")
print("="*80)
print(results_df.to_string(index=False))

# Save results
results_df.to_csv(output_dir / 'model_comparison.csv', index=False)
logger.info("✓ Results saved to CSV")

In [ ]:
# Comparison bar plots
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

metrics = ['Test_MAE', 'Test_RMSE', 'Test_R2']
titles = ['Mean Absolute Error (Test)', 'Root Mean Squared Error (Test)', 'R² Score (Test)']

for idx, (metric, title) in enumerate(zip(metrics, titles)):
    ax = axes[idx]
    results_df.plot(x='Model', y=metric, kind='bar', ax=ax, legend=False, color='steelblue', edgecolor='black')
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_ylabel(metric.replace('_', ' '), fontsize=12)
    ax.set_xlabel('Model', fontsize=12)
    ax.grid(axis='y', alpha=0.3)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.savefig(output_dir / 'model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
logger.info("✓ Model comparison plot saved")

In [ ]:
# Prediction vs Actual for all models
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.flatten()

for idx, (model_name, (model, predictions)) in enumerate(models.items()):
    ax = axes[idx]
    ax.scatter(y_test, predictions, alpha=0.5, s=30)
    
    # Perfect prediction line
    min_val = min(y_test.min(), predictions.min())
    max_val = max(y_test.max(), predictions.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
    
    r2 = r2_score(y_test, predictions)
    ax.set_title(f'{model_name}\nR² = {r2:.4f}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Actual Frequency (Hz)', fontsize=11)
    ax.set_ylabel('Predicted Frequency (Hz)', fontsize=11)
    ax.legend()
    ax.grid(alpha=0.3)

# Hide last subplot
axes[5].axis('off')

plt.tight_layout()
plt.savefig(output_dir / 'prediction_vs_actual.png', dpi=300, bbox_inches='tight')
plt.show()
logger.info("✓ Prediction vs actual plot saved")

## 8. Residual Analysis

In [ ]:
# Residual plots for top 3 models
top_models = results_df.nlargest(3, 'Test_R2')['Model'].tolist()

fig, axes = plt.subplots(1, 3, figsize=(20, 5))

for idx, model_name in enumerate(top_models):
    model, predictions = models[model_name]
    residuals = y_test - predictions
    
    ax = axes[idx]
    ax.scatter(predictions, residuals, alpha=0.5, s=30)
    ax.axhline(y=0, color='r', linestyle='--', lw=2)
    ax.set_title(f'{model_name}', fontsize=13, fontweight='bold')
    ax.set_xlabel('Predicted Frequency (Hz)', fontsize=11)
    ax.set_ylabel('Residuals (Hz)', fontsize=11)
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'residual_plots.png', dpi=300, bbox_inches='tight')
plt.show()
logger.info("✓ Residual plots saved")

## 9. Feature Importance Analysis

In [ ]:
# Feature importance for tree-based models
tree_models = ['Random Forest', 'XGBoost', 'CatBoost']

fig, axes = plt.subplots(1, 3, figsize=(20, 5))

for idx, model_name in enumerate(tree_models):
    model, _ = models[model_name]
    
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    else:
        importances = model.get_feature_importance()
    
    ax = axes[idx]
    y_pos = np.arange(len(feature_cols))
    ax.barh(y_pos, importances, edgecolor='black')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(feature_cols)
    ax.set_xlabel('Importance', fontsize=11)
    ax.set_title(f'{model_name} Feature Importance', fontsize=13, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()
logger.info("✓ Feature importance plot saved")

## 10. SHAP Analysis

In [ ]:
# SHAP for best model
best_model_name = results_df.loc[results_df['Test_R2'].idxmax(), 'Model']
best_model, _ = models[best_model_name]

logger.info(f"\nPerforming SHAP analysis for best model: {best_model_name}")

# Create SHAP explainer
if best_model_name in ['Random Forest', 'XGBoost', 'CatBoost']:
    explainer = shap.TreeExplainer(best_model)
    shap_values = explainer.shap_values(X_test_scaled)
    
    # Summary plot
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X_test_scaled, feature_names=feature_cols, show=False)
    plt.title(f'SHAP Summary Plot - {best_model_name}', fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig(output_dir / 'shap_summary.png', dpi=300, bbox_inches='tight')
    plt.show()
    logger.info("✓ SHAP summary plot saved")
else:
    logger.info("SHAP analysis not available for this model type")

## 11. Save Best Model

In [ ]:
# Save best model and scaler
model_dir = Path('simulation/models')
model_dir.mkdir(parents=True, exist_ok=True)

joblib.dump(best_model, model_dir / f'best_model_{best_model_name.replace(" ", "_")}.pkl')
joblib.dump(scaler, model_dir / 'scaler.pkl')

logger.info(f"✓ Best model ({best_model_name}) saved to {model_dir}")
logger.info(f"✓ Scaler saved to {model_dir}")

print(f"\n✓ Best model: {best_model_name}")
print(f"✓ Saved to: {model_dir}")

## 12. Prediction Interface

In [ ]:
def predict_frequency(length, width, depth, conc_strength, damage_severity):
    """
    Predict natural frequency for given beam parameters.
    
    Parameters:
    -----------
    length : float - Beam length (m)
    width : float - Beam width (m)
    depth : float - Beam depth (m)
    conc_strength : float - Concrete strength (MPa)
    damage_severity : float - Damage severity (0-100%)
    
    Returns:
    --------
    float - Predicted Mode 1 frequency (Hz)
    """
    # Create input array
    input_data = np.array([[length, width, depth, conc_strength, damage_severity]])
    
    # Scale input
    input_scaled = scaler.transform(input_data)
    
    # Predict
    prediction = best_model.predict(input_scaled)[0]
    
    return prediction

# Test prediction function
test_length = 4.0
test_width = 0.3
test_depth = 0.5
test_strength = 35
test_damage = 10

pred_freq = predict_frequency(test_length, test_width, test_depth, test_strength, test_damage)

print("\n" + "="*80)
print("PREDICTION TEST")
print("="*80)
print(f"Input Parameters:")
print(f"  Length: {test_length} m")
print(f"  Width: {test_width} m")
print(f"  Depth: {test_depth} m")
print(f"  Concrete Strength: {test_strength} MPa")
print(f"  Damage Severity: {test_damage}%")
print(f"\nPredicted Mode 1 Frequency: {pred_freq:.2f} Hz")

## 13. Final Summary

In [ ]:
print("\n" + "="*80)
print("TRAINING SUMMARY")
print("="*80)
print(f"\nDataset Size: {len(df)} samples")
print(f"Training Samples: {len(X_train)}")
print(f"Testing Samples: {len(X_test)}")
print(f"\nFeatures: {', '.join(feature_cols)}")
print(f"\nModels Trained: {len(models)}")
print(f"\nBest Model: {best_model_name}")
best_metrics = results_df[results_df['Model'] == best_model_name].iloc[0]
print(f"  Test R²: {best_metrics['Test_R2']:.4f}")
print(f"  Test MAE: {best_metrics['Test_MAE']:.4f} Hz")
print(f"  Test RMSE: {best_metrics['Test_RMSE']:.4f} Hz")

logger.info("\n" + "="*80)
logger.info("ML TRAINING COMPLETED SUCCESSFULLY")
logger.info("="*80)
logger.info(f"Best Model: {best_model_name} (R² = {best_metrics['Test_R2']:.4f})")